# Ejercicio de Ozono

In [1]:
import pandas as pd 
import numpy as np

## Leyendo datos y creando dataframe

In [2]:
df = pd.read_csv('ozono.txt', header = None)
df[['Date','Hour','O3']] = df.pop(0).str.split(n=2, expand=True)
df.head(10)


,Date,Hour,O3
0,000101,0:00,32.80
1,000101,1:00,34.80
2,000101,2:00,36.50
3,000101,3:00,31.60
4,000101,4:00,34.50
5,000101,5:00,30.60
6,000101,6:00,32.00
7,000101,7:00,32.60
8,000101,8:00,35.80
9,000101,9:00,42.10


## Restructurando columnas 

In [3]:
df['Date'] = df['Date'].astype(str) 
df['Year'] = df['Date'].str[0:2]
df['Month'] = df['Date'].str[2:4]
df['Day'] = df['Date'].str[4:]


In [4]:
def year(x):
    x = '20' + str(x)
    return x
df['Date'] = df['Date'].apply(lambda x: year(x))
df['Year'] = df['Year'].apply(lambda x: year(x))


In [5]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')
df['Hour'] = pd.to_datetime(df['Hour'], format='%H:%M')
df['Hour'] =df['Hour'].dt.hour


In [6]:
df['O3'] = round(pd.to_numeric(df['O3'],downcast='float'),2)
x  = list(range(0,26304))
df['index'] = x

## Generando función para calcular la media de 8h

In [7]:
def mean_calc(x):
    count = 0
    if (x+8) in df['index']:
        for value in (range(x, (x+8))):
              count += df['O3'].loc[value]
        m = count/8
    else:
        for value in (range(x, (26304))):
               count += df['O3'].loc[value]
        m = count/ (26304 - x)

    return m

In [8]:
df['O3_8hmean'] = df['index'].apply(lambda x: mean_calc(x))

In [9]:
df_final = df[['Date','Year','Month','Day','Hour','O3_8hmean']]
df_final

,Date,Year,Month,Day,Hour,O3_8hmean
0,2000-01-01,2000,01,01,0,33.175000
1,2000-01-01,2000,01,01,1,33.550000
2,2000-01-01,2000,01,01,2,34.462500
3,2000-01-01,2000,01,01,3,37.050000
4,2000-01-01,2000,01,01,4,41.062500
...,...,...,...,...,...,...
26299,2002-12-31,2002,12,31,19,61.920000
26300,2002-12-31,2002,12,31,20,61.450000
26301,2002-12-31,2002,12,31,21,61.100000
26302,2002-12-31,2002,12,31,22,61.949999


## Exportando archivo a .txt

In [10]:
tfile = open('Ozono8h.txt', 'a')
tfile.write(df_final.to_string())
tfile.close()

## Encontrando max

In [11]:
def find_max(x):
    max_list =[]
    for x in (range(x, x+23)):
        max_list.append(df_final['O3_8hmean'].loc[x])
    return np.max(max_list)

In [12]:
df_final2 = df[['Date','Year','Month','Day','Hour','O3_8hmean']]

In [13]:
df_final2['Year']=df_final2['Date'].dt.year
df_final2['Month']=df_final2['Date'].dt.month
df_final2['Day']=df_final2['Date'].dt.day

<ipython-input-13-f3655f142db5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final2['Year']=df_final2['Date'].dt.year
<ipython-input-13-f3655f142db5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final2['Month']=df_final2['Date'].dt.month
<ipython-input-13-f3655f142db5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [14]:
df_final2.dtypes

Date         datetime64[ns]
Year                  int64
Month                 int64
Day                   int64
Hour                  int64
O3_8hmean           float64
dtype: object

In [15]:
df_grouped=df.pivot_table(index=('Year','Month','Day'),values='O3_8hmean',aggfunc='max')
my_list = []
my_list.append(df_grouped['O3_8hmean'])

In [16]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)
display(my_list)


[Year  Month  Day
 2000  01     01      57.687500
              02      62.675000
              03      55.862500
              04      55.412499
              05     175.500003
                        ...    
 2002  12     27     187.250003
              28      70.537500
              29      58.575000
              30      67.812501
              31      67.675001
 Name: O3_8hmean, Length: 1096, dtype: float64]

In [17]:
with open('Ozono8h_2.txt', 'a') as f:
    for item in my_list:
        f.write("%s\n" % item)

## Percentil 95

In [18]:
df_grouped_sorted = df_grouped.sort_values('O3_8hmean')

In [19]:
Q95 = np.percentile(df_grouped_sorted['O3_8hmean'], 95)
Q95

439.696884393692

In [20]:
df_grouped95 = df_grouped[df_grouped['O3_8hmean']>Q95]

In [21]:
my_list2 = []
my_list2.append(df_grouped95['O3_8hmean'])

In [23]:
l(my_list2)

[Year  Month  Day
 2000  01     12     531.950012
              21     999.900024
              22     999.900024
              23     999.900024
              24     999.900024
       07     13     999.900024
              14     882.900022
              22     999.900024
              23     999.900024
       08     27     999.900024
              28     882.287521
       11     29     530.875012
       12     07     765.512518
              20     522.262512
              21     999.900024
              24     999.900024
              25     999.900024
              26     999.900024
              27     999.900024
 2001  01     05     999.900024
       03     03     999.900024
              04     999.900024
       04     18     520.025012
              19     518.526262
       05     19     653.750014
       06     18     548.487513
              19     999.900024
              20     443.287509
              22     999.900024
              23     882.300021
              24     9

In [22]:
with open('Ozonosup95p.txt', 'a') as f:
    for item in my_list2:
        f.write("%s\n" % item)